### Coordinates Extraction
This notebook is used to match cities to geographical coordinates to be used in Tableau.

In [1]:
# The %... is an iPython thing, and is not part of the Python language.
# In this case we're just telling the plotting library to draw things on
# the notebook, instead of on a separate window.
%matplotlib inline
# See all the "as ..." contructs? They're just aliasing the package names.
# That way we can call methods like plt.plot() instead of matplotlib.pyplot.plot().
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
import json
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")
import random

In [2]:
from pyquery import PyQuery as pq
from bs4 import BeautifulSoup
# The "requests" library makes working with HTTP requests easier
# than the built-in urllib libraries.
import requests

In [3]:
#Get a list of cities with population greater than 1000. This is taken from http://download.geonames.org/export/dump/
citiesraw = []
citiesfile = open('tempdata/cities1000.txt', 'r')
#citiesfile.readline()
while 1:
    line = citiesfile.readline()
    if not line:
        break
    citiesraw.append(line)

In [4]:
#Use largest population to resolve conflict. This may result in some matching errors
print len(citiesraw)
print citiesraw[1]
print citiesraw[1].split('\t')

coordinatesdict = {}
duplicatecities = {}
for x in range(len(citiesraw)):
    templist = citiesraw[x].split('\t')
    cityname = templist[1].lower()
    if cityname not in coordinatesdict:
        coordinatesdict[templist[1].lower()] = (templist[4],templist[5])
        duplicatecities[cityname]=templist[14]
    else:
        if templist[14] < duplicatecities[cityname]:
            duplicatecities[cityname]=templist[14]
            coordinatesdict[templist[1].lower()] = (templist[4],templist[5])
            
    #Code to check current progress
    if x%1000 == 0:
        print x

145493
3039163	Sant Julià de Lòria	Sant Julia de Loria	San Julia,San Julià,Sant Julia de Loria,Sant Julià de Lòria,Sant-Zhulija-de-Lorija,sheng hu li ya-de luo li ya,Сант-Жулия-де-Лория,サン・ジュリア・デ・ロリア教区,圣胡利娅-德洛里亚,圣胡利娅－德洛里亚	42.46372	1.49129	P	PPLA	AD		06				8022		921	Europe/Andorra	2013-11-23

['3039163', 'Sant Juli\xc3\xa0 de L\xc3\xb2ria', 'Sant Julia de Loria', 'San Julia,San Juli\xc3\xa0,Sant Julia de Loria,Sant Juli\xc3\xa0 de L\xc3\xb2ria,Sant-Zhulija-de-Lorija,sheng hu li ya-de luo li ya,\xd0\xa1\xd0\xb0\xd0\xbd\xd1\x82-\xd0\x96\xd1\x83\xd0\xbb\xd0\xb8\xd1\x8f-\xd0\xb4\xd0\xb5-\xd0\x9b\xd0\xbe\xd1\x80\xd0\xb8\xd1\x8f,\xe3\x82\xb5\xe3\x83\xb3\xe3\x83\xbb\xe3\x82\xb8\xe3\x83\xa5\xe3\x83\xaa\xe3\x82\xa2\xe3\x83\xbb\xe3\x83\x87\xe3\x83\xbb\xe3\x83\xad\xe3\x83\xaa\xe3\x82\xa2\xe6\x95\x99\xe5\x8c\xba,\xe5\x9c\xa3\xe8\x83\xa1\xe5\x88\xa9\xe5\xa8\x85-\xe5\xbe\xb7\xe6\xb4\x9b\xe9\x87\x8c\xe4\xba\x9a,\xe5\x9c\xa3\xe8\x83\xa1\xe5\x88\xa9\xe5\xa8\x85\xef\xbc\x8d\xe5\xbe\xb7\xe6\xb4\x9b\xe9\x8

In [5]:
#Get a dictionary of states in the US. We are removing these.
states = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}

In [6]:
stateslist = [state.lower() for state in states.values()]
print len(coordinatesdict)
for state in stateslist:
    if state in coordinatesdict:
        del coordinatesdict[state]
print len(coordinatesdict)

del coordinatesdict['uk']

125626
125610


In [7]:
#Get a list of country names
import codecs
cities2raw = []
with codecs.open('tempdata/CountriesToCities.json', 'r', encoding= "UTF-16") as f:
    for line in f:
       cities2raw.append(json.loads(line))

In [8]:
countries = [item.lower() for item in cities2raw[0].keys()]

In [9]:
#We are removing these from the cities list too. This is because there are some cities that have names of countries.
#Eg There is a city called "China" in Texas. To reduce errors, we remove these cities as well
print len(coordinatesdict)
for country in countries:
    if country in coordinatesdict:
        del coordinatesdict[country]
print len(coordinatesdict)

125609
125561


In [11]:
#Get the raw data files
df = pd.read_json("tempdata/movieinfo_2014_full.json")
df = df.append(pd.read_json("tempdata/movieinfo_2013.json"))
df = df.append(pd.read_json("tempdata/movieinfo_2012.json"))
df = df.append(pd.read_json("tempdata/movieinfo_2011.json"))
df = df.append(pd.read_json("tempdata/movieinfo_2010.json"))
df = df.append(pd.read_json("tempdata/movieinfo_2009.json"))

In [12]:
#Reset the index
r_df = df[~df["user_ratings"].isnull()].reset_index()

In [13]:
len(r_df)

32788

In [14]:
#Remove entries without location data. we are not using these in both our data analysis and visualization
new_df = r_df[r_df["location_page"]!="None"].reset_index()
new_df

,level_0,index,budget,contentRating,country,critic_ratings,duration,genre,gross,language,location_page,name,opening_weekend,release_dates,url,user_ratings,user_ratings_count,year
0,0,0,"€1,500,000",NaN,Kazakhstan,NaN,110 min,[Drama],NaN,Russian,"[Minsk, Belarus , Almaty, Kazakhstan , St. Pet...",Ya ne vernus,NaN,[ 1 March 2014 (Russia) ],/title/tt2637844/,6.9,320,2014
1,2,3,NaN,NaN,USA,NaN,85 min,"[Horror, Mystery, Thriller]",NaN,English,"[Silt, Colorado, USA ]",Find Me,NaN,[ 1 September 2014 (USA) ],/title/tt3027188/,4.5,649,2014
2,3,4,NaN,NaN,Ireland,NaN,88 min,"[Comedy, Drama, Family]",NaN,English,"[Dublin, County Dublin, Ireland , County Wickl...",Gold,NaN,[ 10 October 2014 (Ireland) ],/title/tt3134422/,6.1,406,2014
3,7,10,NaN,Not,USA,NaN,85 min,"[Action, Sci-Fi, Thriller]",NaN,English,"[Los Angeles, California, USA , Long Beach, Ca...",Mega Shark vs. Mecha Shark,NaN,[ 28 January 2014 (USA) ],/title/tt3152098/,2.6,"1,988",2014
4,8,11,NaN,NaN,Poland,NaN,117 min,"[Drama, Romance]",NaN,Polish,"[Ancona, Marche, Italy , Warsaw, Mazowieckie, ...",Obce cialo,NaN,[ 5 December 2014 (Poland) ],/title/tt3997248/,4.6,156,2014
5,12,19,"$50,000",NaN,USA,NaN,NaN,[Comedy],NaN,English,"[Angelus Oaks, California, USA , California, U...",Camp-Off,NaN,[ 4 April 2014 (USA) ],/title/tt3482042/,7.3,7,2014
6,14,21,NaN,NaN,Germany,NaN,84 min,"[Comedy, Drama]",NaN,German,"[Akademie der Künste, Berlin, Germany , Haus d...",Ich will mich nicht künstlich aufregen,NaN,[ February 2014 (Germany) ],/title/tt3471498/,6.2,28,2014
7,15,23,NaN,NaN,Sweden,NaN,99 min,"[Action, Crime, Drama]",NaN,Swedish,"[Falkenberg, Hallands län, Sweden , Uddevalla,...",Svart kung,NaN,[ 15 March 2014 (Sweden) ],/title/tt2935416/,6.2,42,2014
8,16,24,"$7,100",NaN,USA,NaN,92 min,[Action],NaN,English,"[New York, USA , Paulsboro, New Jersey, USA , ...",Battle,NaN,[ 25 December 2014 (USA) ],/title/tt2094769/,6.6,8,2014
9,20,29,NaN,NaN,UK,NaN,90 min,[Horror],NaN,English,"[Wales, UK ]",Valley of the Witch,NaN,[ 13 January 2015 (USA) ],/title/tt2908340/,5.0,67,2014


In [15]:
len(new_df)

18031

In [16]:
#Seperate the locations by commas. 
places = set()
places_freq = {}
separated_places = []
for row in new_df["location_page"]:
    temp = []
    for loc in row:
        split = loc.split(",")
        for item in split:
            stripped = item.encode('utf8').strip().lower()
            if stripped!="":
                places.add(stripped)
                temp.append(stripped)
                if stripped not in places_freq.keys():
                    places_freq[stripped] = 1
                places_freq[stripped] = places_freq[stripped] + 1
    separated_places.append(temp)
df_boolean = new_df.copy()
df_boolean['separated_places'] = separated_places

In [17]:
len(places)

18983

In [18]:
places_freq = []
separated_places = []
for row in new_df["location_page"]:
    temp = []
    for loc in row:
        split = loc.split(",")
        for item in split:
            stripped = item.encode('utf8').strip().lower()
            if stripped!="":
                places.add(stripped)
                temp.append(stripped)
    separated_places.append(temp)
df_boolean = new_df.copy()
df_boolean['separated_places'] = separated_places

In [19]:
genreset = set()

for index, row in new_df.iterrows():
    genreslist = row.genre
    for genre in genreslist:
        genreset.add(genre.encode('utf8'))

In [20]:
print len(genreset)
genreset

24


{'Action',
 'Adult',
 'Adventure',
 'Animation',
 'Biography',
 'Comedy',
 'Crime',
 'Drama',
 'Family',
 'Fantasy',
 'Game-Show',
 'History',
 'Horror',
 'Music',
 'Musical',
 'Mystery',
 'News',
 'Reality-TV',
 'Romance',
 'Sci-Fi',
 'Sport',
 'Thriller',
 'War',
 'Western'}

In [21]:
df_boolean.head()

,level_0,index,budget,contentRating,country,critic_ratings,duration,genre,gross,language,location_page,name,opening_weekend,release_dates,url,user_ratings,user_ratings_count,year,separated_places
0,0,0,"€1,500,000",NaN,Kazakhstan,NaN,110 min,[Drama],NaN,Russian,"[Minsk, Belarus , Almaty, Kazakhstan , St. Pet...",Ya ne vernus,NaN,[ 1 March 2014 (Russia) ],/title/tt2637844/,6.9,320,2014,"[minsk, belarus, almaty, kazakhstan, st. peter..."
1,2,3,NaN,NaN,USA,NaN,85 min,"[Horror, Mystery, Thriller]",NaN,English,"[Silt, Colorado, USA ]",Find Me,NaN,[ 1 September 2014 (USA) ],/title/tt3027188/,4.5,649,2014,"[silt, colorado, usa]"
2,3,4,NaN,NaN,Ireland,NaN,88 min,"[Comedy, Drama, Family]",NaN,English,"[Dublin, County Dublin, Ireland , County Wickl...",Gold,NaN,[ 10 October 2014 (Ireland) ],/title/tt3134422/,6.1,406,2014,"[dublin, county dublin, ireland, county wicklo..."
3,7,10,NaN,Not,USA,NaN,85 min,"[Action, Sci-Fi, Thriller]",NaN,English,"[Los Angeles, California, USA , Long Beach, Ca...",Mega Shark vs. Mecha Shark,NaN,[ 28 January 2014 (USA) ],/title/tt3152098/,2.6,"1,988",2014,"[los angeles, california, usa, long beach, cal..."
4,8,11,NaN,NaN,Poland,NaN,117 min,"[Drama, Romance]",NaN,Polish,"[Ancona, Marche, Italy , Warsaw, Mazowieckie, ...",Obce cialo,NaN,[ 5 December 2014 (Poland) ],/title/tt3997248/,4.6,156,2014,"[ancona, marche, italy, warsaw, mazowieckie, p..."


In [23]:
#Match the cities to the dictionary of coordinates
coordinateslist = []
coordinates_cities = []

for entry in df_boolean['separated_places']:


    coordinateslist.append([coordinatesdict[x] for x in entry if x in coordinatesdict])
    coordinates_cities.append([x for x in entry if x in coordinatesdict])


    
df_boolean['coordinates'] = coordinateslist
df_boolean['coordinates_cities'] = coordinates_cities

In [24]:
df_boolean.head()

,level_0,index,budget,contentRating,country,critic_ratings,duration,genre,gross,language,location_page,name,opening_weekend,release_dates,url,user_ratings,user_ratings_count,year,separated_places,coordinates,coordinates_cities
0,0,0,"€1,500,000",NaN,Kazakhstan,NaN,110 min,[Drama],NaN,Russian,"[Minsk, Belarus , Almaty, Kazakhstan , St. Pet...",Ya ne vernus,NaN,[ 1 March 2014 (Russia) ],/title/tt2637844/,6.9,320,2014,"[minsk, belarus, almaty, kazakhstan, st. peter...","[(53.9, 27.56667), (43.25654, 76.92848)]","[minsk, almaty]"
1,2,3,NaN,NaN,USA,NaN,85 min,"[Horror, Mystery, Thriller]",NaN,English,"[Silt, Colorado, USA ]",Find Me,NaN,[ 1 September 2014 (USA) ],/title/tt3027188/,4.5,649,2014,"[silt, colorado, usa]","[(39.54859, -107.65617)]",[silt]
2,3,4,NaN,NaN,Ireland,NaN,88 min,"[Comedy, Drama, Family]",NaN,English,"[Dublin, County Dublin, Ireland , County Wickl...",Gold,NaN,[ 10 October 2014 (Ireland) ],/title/tt3134422/,6.1,406,2014,"[dublin, county dublin, ireland, county wicklo...","[(53.33306, -6.24889)]",[dublin]
3,7,10,NaN,Not,USA,NaN,85 min,"[Action, Sci-Fi, Thriller]",NaN,English,"[Los Angeles, California, USA , Long Beach, Ca...",Mega Shark vs. Mecha Shark,NaN,[ 28 January 2014 (USA) ],/title/tt3152098/,2.6,"1,988",2014,"[los angeles, california, usa, long beach, cal...","[(34.05223, -118.24368), (41.73893, -86.85697)]","[los angeles, long beach]"
4,8,11,NaN,NaN,Poland,NaN,117 min,"[Drama, Romance]",NaN,Polish,"[Ancona, Marche, Italy , Warsaw, Mazowieckie, ...",Obce cialo,NaN,[ 5 December 2014 (Poland) ],/title/tt3997248/,4.6,156,2014,"[ancona, marche, italy, warsaw, mazowieckie, p...","[(43.5942, 13.50337), (41.2381, -85.85305), (5...","[ancona, warsaw, moscow]"


In [ ]:
df_boolean.to_csv('df_boolean_temp3.csv',sep=',',encoding='utf-8')

In [45]:
#After using df_boolean_temp3 in tableau, we realized that Tableau needs exactly 1 coordinate per entry. 
#The multiple coordinates per entry is confusing Tableau. Hence, we have to flatten it

old_df = df_boolean.copy()
del old_df['coordinates']
del old_df['coordinates_cities']
new_df = pd.DataFrame(columns = df_boolean.columns)


    

In [46]:
old_df.head()

,level_0,index,budget,contentRating,country,critic_ratings,duration,genre,gross,language,location_page,name,opening_weekend,release_dates,url,user_ratings,user_ratings_count,year,separated_places
0,0,0,"€1,500,000",NaN,Kazakhstan,NaN,110 min,[Drama],NaN,Russian,"[Minsk, Belarus , Almaty, Kazakhstan , St. Pet...",Ya ne vernus,NaN,[ 1 March 2014 (Russia) ],/title/tt2637844/,6.9,320,2014,"[minsk, belarus, almaty, kazakhstan, st. peter..."
1,2,3,NaN,NaN,USA,NaN,85 min,"[Horror, Mystery, Thriller]",NaN,English,"[Silt, Colorado, USA ]",Find Me,NaN,[ 1 September 2014 (USA) ],/title/tt3027188/,4.5,649,2014,"[silt, colorado, usa]"
2,3,4,NaN,NaN,Ireland,NaN,88 min,"[Comedy, Drama, Family]",NaN,English,"[Dublin, County Dublin, Ireland , County Wickl...",Gold,NaN,[ 10 October 2014 (Ireland) ],/title/tt3134422/,6.1,406,2014,"[dublin, county dublin, ireland, county wicklo..."
3,7,10,NaN,Not,USA,NaN,85 min,"[Action, Sci-Fi, Thriller]",NaN,English,"[Los Angeles, California, USA , Long Beach, Ca...",Mega Shark vs. Mecha Shark,NaN,[ 28 January 2014 (USA) ],/title/tt3152098/,2.6,"1,988",2014,"[los angeles, california, usa, long beach, cal..."
4,8,11,NaN,NaN,Poland,NaN,117 min,"[Drama, Romance]",NaN,Polish,"[Ancona, Marche, Italy , Warsaw, Mazowieckie, ...",Obce cialo,NaN,[ 5 December 2014 (Poland) ],/title/tt3997248/,4.6,156,2014,"[ancona, marche, italy, warsaw, mazowieckie, p..."


In [51]:
#WARNING: THIS CELL TAKES A LONG TIME TO RUN. 
#Approximately 1 to 2 hours
#For visualization in Tableau, we want each entry to have only 1 location. so we have to flatten the dataframe based on locations


flattencoordinates = []
flattencoordinatescities = []
newdfcounter = 0
print len(df_boolean)
for x in range(len(df_boolean)):
    for y in range(len(df_boolean['coordinates'][x])):
        new_df.loc[newdfcounter] = old_df.iloc[x]
        flattencoordinates.append(df_boolean['coordinates'][x][y]) 
        flattencoordinatescities.append(df_boolean['coordinates_cities'][x][y]) 
        newdfcounter = newdfcounter + 1
    
    #Printing to keep track of current progress
    if x%100 == 0:
        print str(x) + "   " + str(newdfcounter)
new_df['coordinates'] = flattencoordinates
new_df['coordinates_cities'] = flattencoordinatescities

18031
0   2
100   191
200   397
300   658
400   870
500   1128
600   1375
700   1632
800   1830
900   2029
1000   2218
1100   2463
1200   2700
1300   2960
1400   3195
1500   3461
1600   3718
1700   4008
1800   4223
1900   4461
2000   4741
2100   4919
2200   5165
2300   5365
2400   5610
2500   5819
2600   6019
2700   6247
2800   6478
2900   6687
3000   6913
3100   7136
3200   7396
3300   7654
3400   7881
3500   8140
3600   8341
3700   8587
3800   8805
3900   9019
4000   9286
4100   9512
4200   9755
4300   10053
4400   10317
4500   10499
4600   10697
4700   10939
4800   11175
4900   11440
5000   11647
5100   11896
5200   12075
5300   12313
5400   12532
5500   12762
5600   12994
5700   13194
5800   13506
5900   13765
6000   13978
6100   14237
6200   14483
6300   14729
6400   14957
6500   15184
6600   15465
6700   15689
6800   15920
6900   16194
7000   16401
7100   16646
7200   16877
7300   17156
7400   17391
7500   17637
7600   17920
7700   18136
7800   18348
7900   18610
8000   18874
810

In [49]:
new_df.head(10)

,level_0,index,budget,contentRating,country,critic_ratings,duration,genre,gross,language,location_page,name,opening_weekend,release_dates,url,user_ratings,user_ratings_count,year,separated_places,coordinates,coordinates_cities
0,0,0,"€1,500,000",NaN,Kazakhstan,NaN,110 min,[Drama],NaN,Russian,"[Minsk, Belarus , Almaty, Kazakhstan , St. Pet...",Ya ne vernus,NaN,[ 1 March 2014 (Russia) ],/title/tt2637844/,6.9,320,2014,"[minsk, belarus, almaty, kazakhstan, st. peter...","(53.9, 27.56667)",minsk
1,0,0,"€1,500,000",NaN,Kazakhstan,NaN,110 min,[Drama],NaN,Russian,"[Minsk, Belarus , Almaty, Kazakhstan , St. Pet...",Ya ne vernus,NaN,[ 1 March 2014 (Russia) ],/title/tt2637844/,6.9,320,2014,"[minsk, belarus, almaty, kazakhstan, st. peter...","(43.25654, 76.92848)",almaty
2,2,3,NaN,NaN,USA,NaN,85 min,"[Horror, Mystery, Thriller]",NaN,English,"[Silt, Colorado, USA ]",Find Me,NaN,[ 1 September 2014 (USA) ],/title/tt3027188/,4.5,649,2014,"[silt, colorado, usa]","(39.54859, -107.65617)",silt
3,3,4,NaN,NaN,Ireland,NaN,88 min,"[Comedy, Drama, Family]",NaN,English,"[Dublin, County Dublin, Ireland , County Wickl...",Gold,NaN,[ 10 October 2014 (Ireland) ],/title/tt3134422/,6.1,406,2014,"[dublin, county dublin, ireland, county wicklo...","(53.33306, -6.24889)",dublin
4,7,10,NaN,Not,USA,NaN,85 min,"[Action, Sci-Fi, Thriller]",NaN,English,"[Los Angeles, California, USA , Long Beach, Ca...",Mega Shark vs. Mecha Shark,NaN,[ 28 January 2014 (USA) ],/title/tt3152098/,2.6,"1,988",2014,"[los angeles, california, usa, long beach, cal...","(34.05223, -118.24368)",los angeles
5,7,10,NaN,Not,USA,NaN,85 min,"[Action, Sci-Fi, Thriller]",NaN,English,"[Los Angeles, California, USA , Long Beach, Ca...",Mega Shark vs. Mecha Shark,NaN,[ 28 January 2014 (USA) ],/title/tt3152098/,2.6,"1,988",2014,"[los angeles, california, usa, long beach, cal...","(41.73893, -86.85697)",long beach
6,8,11,NaN,NaN,Poland,NaN,117 min,"[Drama, Romance]",NaN,Polish,"[Ancona, Marche, Italy , Warsaw, Mazowieckie, ...",Obce cialo,NaN,[ 5 December 2014 (Poland) ],/title/tt3997248/,4.6,156,2014,"[ancona, marche, italy, warsaw, mazowieckie, p...","(43.5942, 13.50337)",ancona
7,8,11,NaN,NaN,Poland,NaN,117 min,"[Drama, Romance]",NaN,Polish,"[Ancona, Marche, Italy , Warsaw, Mazowieckie, ...",Obce cialo,NaN,[ 5 December 2014 (Poland) ],/title/tt3997248/,4.6,156,2014,"[ancona, marche, italy, warsaw, mazowieckie, p...","(41.2381, -85.85305)",warsaw
8,8,11,NaN,NaN,Poland,NaN,117 min,"[Drama, Romance]",NaN,Polish,"[Ancona, Marche, Italy , Warsaw, Mazowieckie, ...",Obce cialo,NaN,[ 5 December 2014 (Poland) ],/title/tt3997248/,4.6,156,2014,"[ancona, marche, italy, warsaw, mazowieckie, p...","(55.75222, 37.61556)",moscow


In [ ]:
new_df.to_csv('df_boolean_temp4.csv',sep=',',encoding='utf-8')